# Step 1 - automated data ingestion

In [3]:
import requests
from datetime import datetime, timedelta
from os import environ
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import plotly.express as px
from sqlalchemy.engine.url import URL
from sqlalchemy_utils import database_exists, create_database, drop_database

api_key = environ.get('aviation_key')
api_url = 'https://api.aviationstack.com/v1/flights'

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

def fetch_flights_for_date(flight_date, airline):
    params = {
        'access_key': api_key,
        'flight_date': flight_date.strftime('%Y-%m-%d'),
        'airline_name': airline
    }
    
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        data = response.json().get('data', [])
        return data
    else:
        print(f"Failed to fetch data for {flight_date}: {response.status_code}")
        return []

def fetch_flights(airline, start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    all_flights = []

    for single_date in daterange(start_date, end_date):
        flights_on_date = fetch_flights_for_date(single_date, airline)
        all_flights.extend(flights_on_date)

    if all_flights:
        return pd.DataFrame(all_flights)
    else:
        return "No data available for the given parameters" 



In [6]:
airline = "Scoot"  
start_date = "2024-01-15"  
end_date = "2024-01-20"  

df = fetch_flights(airline, start_date, end_date)
df.head()

,flight_date,flight_status,departure,arrival,airline,flight,aircraft,live
0,2024-01-19,landed,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'airport': 'Penang International', 'timezone'...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '424', 'iata': 'TR424', 'icao': 'TG...",None,None
1,2024-01-19,landed,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'airport': 'Ninoy Aquino International', 'tim...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '390', 'iata': 'TR390', 'icao': 'TG...","{'registration': '9V-NCF', 'iata': 'A21N', 'ic...",None
2,2024-01-19,landed,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'airport': 'Nanchang', 'timezone': 'Asia/Shan...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '112', 'iata': 'TR112', 'icao': 'TG...",None,None
3,2024-01-19,scheduled,"{'airport': 'Singapore Changi', 'timezone': 'A...",{'airport': 'Sultan Aji Muhamad Sulaiman Airpo...,"{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '222', 'iata': 'TR222', 'icao': 'TG...",None,None
4,2024-01-19,landed,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'airport': 'Krabi', 'timezone': 'Asia/Bangkok...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '686', 'iata': 'TR686', 'icao': 'TG...",None,None


In [82]:
depart_tb = pd.DataFrame(df['departure'].to_list(), columns = ['airport', 'timezone','iata','icao',
                                                          'terminal','gate','delay','scheduled',
                                                          'estimated','actual','estimated_runway','actual_runway'])

arrival_tb = pd.DataFrame(df['arrival'].to_list(), columns = ['airport', 'timezone','iata','icao',
                                                          'terminal','gate','baggage','delay','scheduled',
                                                          'estimated','actual','estimated_runway','actual_runway'])

airline_tb = pd.DataFrame(df['airline'].to_list(), columns = ['name','iata','icao'])

flight_tb = pd.DataFrame(df['flight'].to_list(), columns = ['number','iata','icao','codeshared'])



aircraft_tb = pd.DataFrame(
    list(
        filter(
            lambda x:x != None,df['aircraft'].to_list()
        )
    ), columns = ['registration','iata','icao','icao24'])



In [97]:
 depart_tb['id'] = depart_tb['iata'].fillna('') + \
                            depart_tb['scheduled'].apply(lambda x: x if pd.notnull(x) else '') + \
                            depart_tb['estimated'].apply(lambda x: x if pd.notnull(x) else '') + \
                            depart_tb['actual'].apply(lambda x: x if pd.notnull(x) else '')

arrival_tb['id'] = arrival_tb['iata'].fillna('') + \
                            arrival_tb['scheduled'].apply(lambda x: x if pd.notnull(x) else '') + \
                            arrival_tb['estimated'].apply(lambda x: x if pd.notnull(x) else '') + \
                            arrival_tb['actual'].apply(lambda x: x if pd.notnull(x) else '')

In [94]:
#table used only for the id creation at df table
aircraft_aux = [{
    'registration': aircraft.get('registration') if aircraft else None,
    'iata': aircraft.get('iata') if aircraft else None,
    'icao': aircraft.get('icao') if aircraft else None,
    'icao24': aircraft.get('icao24') if aircraft else None
} for aircraft in df['aircraft']]

# Convert the list of dictionaries to a DataFrame
aircraft_aux = pd.DataFrame(aircraft_aux)

In [95]:
df['dpt_id'] = depart_tb['iata'].fillna('') + \
                            depart_tb['scheduled'].apply(lambda x: x if pd.notnull(x) else '') + \
                            depart_tb['estimated'].apply(lambda x: x if pd.notnull(x) else '') + \
                            depart_tb['actual'].apply(lambda x: x if pd.notnull(x) else '')
df['arr_id'] = arrival_tb['iata'].fillna('') + \
                            arrival_tb['scheduled'].apply(lambda x: x if pd.notnull(x) else '') + \
                            arrival_tb['estimated'].apply(lambda x: x if pd.notnull(x) else '') + \
                            arrival_tb['actual'].apply(lambda x: x if pd.notnull(x) else '')
df['airline_id'] = airline_tb['iata']
df['flight_id'] = flight_tb['iata']
df['aircraft_id'] = aircraft_aux['iata']
df['id'] = df['flight_id'] + df['flight_date'].str.replace('-', '')

,flight_date,flight_status,departure,arrival,airline,flight,aircraft,live,dpt_id,arr_id,airline_id,flight_id,aircraft_id
0,2024-01-19,landed,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'airport': 'Penang International', 'timezone'...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '424', 'iata': 'TR424', 'icao': 'TG...",None,None,SIN2024-01-19T14:55:00+00:002024-01-19T14:55:0...,PEN2024-01-19T16:20:00+00:002024-01-19T16:20:0...,TR,TR424,None
1,2024-01-19,landed,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'airport': 'Ninoy Aquino International', 'tim...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '390', 'iata': 'TR390', 'icao': 'TG...","{'registration': '9V-NCF', 'iata': 'A21N', 'ic...",None,SIN2024-01-19T14:50:00+00:002024-01-19T14:50:0...,MNL2024-01-19T18:30:00+00:002024-01-19T18:30:0...,TR,TR390,A21N
2,2024-01-19,landed,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'airport': 'Nanchang', 'timezone': 'Asia/Shan...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '112', 'iata': 'TR112', 'icao': 'TG...",None,None,SIN2024-01-19T14:30:00+00:002024-01-19T14:30:0...,KHN2024-01-19T19:15:00+00:002024-01-19T19:15:0...,TR,TR112,None
3,2024-01-19,scheduled,"{'airport': 'Singapore Changi', 'timezone': 'A...",{'airport': 'Sultan Aji Muhamad Sulaiman Airpo...,"{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '222', 'iata': 'TR222', 'icao': 'TG...",None,None,SIN2024-01-19T14:15:00+00:002024-01-19T14:15:0...,BPN2024-01-19T16:40:00+00:002024-01-19T16:40:0...,TR,TR222,None
4,2024-01-19,landed,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'airport': 'Krabi', 'timezone': 'Asia/Bangkok...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '686', 'iata': 'TR686', 'icao': 'TG...",None,None,SIN2024-01-19T14:25:00+00:002024-01-19T14:25:0...,KBV2024-01-19T15:25:00+00:002024-01-19T15:25:0...,TR,TR686,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2024-01-20,landed,"{'airport': 'Seoul (Incheon)', 'timezone': 'As...","{'airport': 'Singapore Changi', 'timezone': 'A...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '841', 'iata': 'TR841', 'icao': 'TG...","{'registration': '9V-OFC', 'iata': 'B788', 'ic...",None,ICN2024-01-20T22:35:00+00:002024-01-20T22:35:0...,SIN2024-01-21T04:10:00+00:002024-01-21T04:10:0...,TR,TR841,B788
196,2024-01-20,landed,"{'airport': 'Ninoy Aquino International', 'tim...","{'airport': 'Singapore Changi', 'timezone': 'A...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '391', 'iata': 'TR391', 'icao': 'TG...","{'registration': '9V-NCF', 'iata': 'A21N', 'ic...",None,MNL2024-01-20T19:30:00+00:002024-01-20T19:30:0...,SIN2024-01-20T23:30:00+00:002024-01-20T23:30:0...,TR,TR391,A21N
197,2024-01-20,landed,{'airport': 'Nanjing Lukou International Airpo...,"{'airport': 'Singapore Changi', 'timezone': 'A...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '181', 'iata': 'TR181', 'icao': 'TG...","{'registration': '9V-OFE', 'iata': 'B788', 'ic...",None,NKG2024-01-20T17:00:00+00:002024-01-20T17:00:0...,SIN2024-01-20T22:35:00+00:002024-01-20T22:35:0...,TR,TR181,B788
198,2024-01-20,landed,"{'airport': 'Hangzhou', 'timezone': 'Asia/Shan...","{'airport': 'Singapore Changi', 'timezone': 'A...","{'name': 'Scoot', 'iata': 'TR', 'icao': 'TGW'}","{'number': '189', 'iata': 'TR189', 'icao': 'TG...","{'registration': '9V-OFI', 'iata': 'B788', 'ic...",None,HGH2024-01-20T23:15:00+00:002024-01-20T23:15:0...,SIN2024-01-21T05:00:00+00:002024-01-21T05:00:0...,TR,TR189,B788


# Step 2 - Data storage


In [140]:
# column live was removed because had ~95% NULLs
df.drop(['departure','arrival','airline','flight','aircraft','live'], axis = 1).to_csv('data/flights_main.csv', index=False)
depart_tb.to_csv('data/dim_dpt.csv', index=False)
arrival_tb.to_csv('data/dim_arr.csv', index=False)
airline_tb.to_csv('data/dim_airline.csv', index=False)
flight_tb.to_csv('data/dim_flight.csv', index=False)
aircraft_tb.to_csv('data/dim_aircraft.csv', index=False)


# Step 3 - Data Querying and filtering

In [7]:
def query_data(engine, airline=None, origin=None, destination=None):
    query = "SELECT * FROM flights WHERE 1=1"
    if airline:
        query += f" AND airline = '{airline}'"
    if origin:
        query += f" AND origin = '{origin}'"
    if destination:
        query += f" AND destination = '{destination}'"
    
    return pd.read_sql(query, con=engine)


# Step 4 - Analyzing the Data

In [8]:
def analyze_data(df):
    total_flights = len(df)
    average_duration = df['duration'].mean()
    flights_per_airline = df['airline'].value_counts()
    return total_flights, average_duration, flights_per_airline


# Step 5 - Visualizations

In [9]:
def plot_data(df):
    # Using Matplotlib for a simple histogram
    plt.figure(figsize=(10, 5))
    plt.hist(df['duration'], bins=20, color='blue')
    plt.title('Flight Duration Distribution')
    plt.xlabel('Duration (Minutes)')
    plt.ylabel('Number of Flights')
    plt.show()

    # Using Plotly for interactive charts
    fig = px.bar(df, x='airline', y='duration', title='Average Flight Duration by Airline')
    fig.show()
